In [114]:
# # install neccesary plugins
# !pip install pandas
# !pip install selenium
# !pip install webdriver_manager
# !pip install bs4

In [25]:
# imports
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.firefox import GeckoDriverManager
import re
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from os import getlogin
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [6]:
# update and run the latest Firefox webdriver
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

[WDM] - ====== WebDriver manager ======


[WDM] - Driver [C:\Users\mn850\.wdm\drivers\geckodriver\win64\v0.29.1\geckodriver.exe] found in cache


In [7]:
# get the html
debank_url = 'https://debank.com/ranking/locked_value'
driver.get(debank_url)
html = driver.page_source

In [10]:
# find all protocols
soup = BeautifulSoup(html, 'lxml')
soup_html = soup.find_all("div", attrs ={'style': 'left: 0px;'})
soup_html[:2]

[<div style="left: 0px;"><img alt="" src="https://static.debank.com/image/project/logo_url/compound/0b792243f1f68e9ed082f5a49ee6f21d.png" style="width: 20px; height: 20px;"/></div>,
 <div style="left: 0px;"><img alt="" src="https://static.debank.com/image/project/logo_url/aave2/b8d1ef85570855176b102edf8cda6f8d.png" style="width: 20px; height: 20px;"/></div>]

In [11]:
# extract protocol name from link list
url_list = []
for url in soup_html:
    url_list.append(re.findall(r'(https?://[^\s]+)', str(url.get_text)))
    
url_list

[['https://static.debank.com/image/project/logo_url/compound/0b792243f1f68e9ed082f5a49ee6f21d.png"'],
 ['https://static.debank.com/image/project/logo_url/aave2/b8d1ef85570855176b102edf8cda6f8d.png"'],
 ['https://static.debank.com/image/project/logo_url/makerdao/3821328c7c6d5ac4fc87e2c2e4d1c684.png"'],
 ['https://static.debank.com/image/project/logo_url/bsc_pancakeswap/a4e035cf4495755fddd5ebb6e5657f63.png"'],
 ['https://static.debank.com/image/project/logo_url/uniswap2/87a541b3b83b041c8d12119e5a0d19f0.png"'],
 ['https://static.debank.com/image/project/logo_url/curve/aa991be165e771cff87ae61e2a61ef68.png"'],
 ['https://static.debank.com/image/project/logo_url/matic_aave/2e117f9e596fa126c58a82315137c7e6.png"'],
 ['https://static.debank.com/image/project/logo_url/sushiswap/248a91277aac1ac16a457b8f61957089.png"'],
 ['https://static.debank.com/image/project/logo_url/liquity/794204ad55475b57b84384f4298e66ba.png"'],
 ['https://static.debank.com/image/project/logo_url/bsc_venus/db05054f7f5306ff2

In [12]:
# convert protocol list to dataframe
df = pd.DataFrame(url_list, columns=['DeBank URL'])
df['DeBank URL'] = df['DeBank URL'].str.replace('"','', regex = True)
df['Protocol'] = df['DeBank URL'].astype(str).str.rsplit('/', n=2, expand = True)[1]
df.head(2)

,DeBank URL,Protocol
0,https://static.debank.com/image/project/logo_url/compound/0b792243f1f68e9ed082f5a49ee6f21d.png,compound
1,https://static.debank.com/image/project/logo_url/aave2/b8d1ef85570855176b102edf8cda6f8d.png,aave2


In [27]:
# obtain Defillama protocols
df_defi_llama = pd.json_normalize(requests.get('https://api.llama.fi/protocols').json(),
                                  max_level = 3)

df_defi_llama.head(2)

,id,name,address,symbol,url,description,chain,logo,audits,audit_note,gecko_id,cmcId,category,chains,module,twitter,slug,tvl,change_1h,change_1d,change_7d,chainTvls.Ethereum,chainTvls.Polygon,chainTvls.Binance,chainTvls.Wanchain,chainTvls.Terra,chainTvls.xDai,chainTvls.OKExChain,chainTvls.Avalanche,chainTvls.Solana,chainTvls.Fantom,chainTvls.Harmony,chainTvls.Heco
0,1,Uniswap,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,https://info.uniswap.org/,A fully decentralized protocol for automated liquidity provision on Ethereum.\r\n,Ethereum,None,2,None,uniswap,7083,Dexes,[Ethereum],uniswap/index.js,Uniswap,uniswap,5.512095e+09,0.425757,-2.417933,-7.90597,5.512095e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,WBTC,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,https://wbtc.network/,Wrapped Bitcoin (WBTC) is the first ERC20 token backed 1:1 with Bitcoin.\r\nCompletely transparent. 100% verifiable. Community led.,Ethereum,https://icons.llama.fi/d2T9nXpn_400x400.png,2,None,wrapped-bitcoin,3717,Assets,[Ethereum],wbtc.js,WrappedBTC,wbtc,6.849200e+09,0.671489,-1.382427,-4.13166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
